<a href="https://colab.research.google.com/github/parra28/ReconocimientoNumero/blob/master/Reconocimiento_de_n%C3%BAmeros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Flujo de trabajo típico del Machine Learning**
La creación de algoritmos de machine learning sigue siempre la misma estructura: 


*   Identificar el problema
*   Identificar el algoritmo adecuado para resolver el problema
*   Recoger suficiente información de **CALIDAD** 
*   Entrenar el modelo
*   Exportar el modelo a un entorno de producción

Hoy vamos a crear un modelo simple para reconocer números escritos a mano.




##**FASE 1: Identificar el problema**
Si pensamos en frio, el problema no es más que una simple clasificación de imágenes. Tenemos que crear un modelo que se alimente a partir de una imagen y nos devuelva la probabilidad de que la imagen que hemos aliementado coincida con una de las categorías que nosotros hemos decidido.


##**FASE 2: Elección del algoritmo**
Los algoritmos por excelencia para la clasificación son las Deep Neural Networks (DNNs). 

La metodología consiste en crear un modelo con unas constantes que, en base a un input, sea capaz de identificar en qué categoría se clasifica. Lo más complicado es el ajuste del modelo, donde debemos identificar qué constantes aproximan mejor la realidad.

Para obtener estas constantes debememos "enseñar" a nuestro modelo qué está bien y qué está mal (de ahí lo de Machine Learning). 

Ahí es donde entra la parte de recoger información de CALIDAD. Contra mejor sea la muestra, mejor va a ser el ajuste.

##**FASE 3: Entrenamiento del modelo**
Importamos los módulos necesarios para entrenar y crear el algoritmo que sea capaz de reconocer los números.

In [ ]:
import cv2 as cv
from google.colab.patches import cv2_imshow
import numpy as np #Librería específica para hacer operaciones vectoriales
import tensorflow as tf

Una vez importadas los módulos, importaremos la base de datos con los números escritos a mano y con su respectiva categorización.

In [ ]:
mnist = tf.keras.datasets.mnist
(xTrain, yTrain),(xTest, yTest) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Ahora veamos a ver qué pinta tiene esto:

In [ ]:
print(f"Vamos a utilizar {xTrain.shape[0]} imágenes para entrenar nuestro modelo y {xTest.shape[0]} para validarlo")
for i, img in enumerate(xTrain[:5]):
  cv2_imshow(img)
  print(yTrain[i]) 


Vamos a utilizar 60000 imágenes para entrenar nuestro modelo y 10000 para validarlo


5


0


4


1


9


Ahora tenemos que normalizar los datos (tenemos que trabajar con 1s y 0s)

In [ ]:
xTrain, xTest = tf.keras.utils.normalize(xTrain, axis=1), tf.keras.utils.normalize(xTest, axis=1)
cv2_imshow(xTrain[0]) #El aspecto que tiene normalizado. 
cv2_imshow(xTrain[0]*255) #Si lo medio rescatamos.
print((xTrain.shape[1:]))

(28, 28)


Definamos el modelo:


In [ ]:
model = tf.keras.models.Sequential() #Una DNN básica
model.add(tf.keras.layers.Flatten(input_shape=(xTrain.shape[1:]))) #Definimos las dimensiones de la matriz que alimentamos al sistema
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu)) #Rectify linear unit if (x < 0) -> x = 0; else x = x
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu)) #Rectify linear unit if (x < 0) -> x = 0; else x = x
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=10, activation=tf.nn.softmax)) #Probabilidades de 0-1 por cada categoría
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Una vez tenemos definido el modelo pasamos a entrenarlo con las imagenes que hemos decidido destinar al entrenamiento

In [ ]:
model.fit(xTrain, yTrain, epochs=3)
loss, accuracy = model.evaluate(xTest, yTest)
print(f"La precisión de nuestro modelo es del {accuracy*100}")

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3508 - accuracy: 0.8968
Epoch 2/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1476 - accuracy: 0.9565
Epoch 3/3
313/313 [==============================] - 1s 1ms/step - loss: 0.1012 - accuracy: 0.9696
La precisión de nuestro modelo es del 96.96000218391418


Ahora pongamos a prueba a nuestro modelo. Para ello vamos a dibujar un número en paint y vamos a alimentarselo a nuestro modelo.


In [ ]:
for i in range(9):
  img = np.array([xTrain[i]])
  cv2_imshow(img[0]*255)
  prediction = model.predict(img)
  print(f"Has dibujado un {np.argmax(prediction)}")

Has dibujado un 5


Has dibujado un 0


Has dibujado un 4


Has dibujado un 1


Has dibujado un 9


Has dibujado un 2


Has dibujado un 1


Has dibujado un 3


Has dibujado un 1
